In [1]:
import os

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 200)

%load_ext autoreload
%autoreload 2

In [2]:
data_filepath = '../data/harddrive.csv'
os.path.exists(data_filepath)

True

## Load dataset to pd DataFrame

In [3]:
df1 = pd.read_csv(data_filepath,
                  usecols=['date', 'serial_number', 'model', 'failure', 'smart_1_raw'])

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179295 entries, 0 to 3179294
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   serial_number  object
 2   model          object
 3   failure        int64 
 4   smart_1_raw    int64 
dtypes: int64(2), object(3)
memory usage: 121.3+ MB


In [7]:
df1.head()

,date,serial_number,model,failure,smart_1_raw
0,2016-01-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,0,0
1,2016-01-01,Z305B2QN,ST4000DM000,0,54551400
2,2016-01-01,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,0,0
3,2016-01-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,0,0
4,2016-01-01,WD-WMC4N2899475,WDC WD30EFRX,0,0


In [8]:
df1['serial_number'].nunique()

65993

In [9]:
df1.groupby(['serial_number'])['date'].count()

serial_number
13H2B97AS    52
13H32WEAS    52
13H6A21GS    52
13H7X2HAS    52
13H85BMGS    52
             ..
Z4D1XY09     52
Z4D29XNF     52
Z4D2B5EC     52
Z4E09K9M     30
Z4E09KB7     30
Name: date, Length: 65993, dtype: int64

In [10]:
df1['failure'].value_counts()

failure
0    3179080
1        215
Name: count, dtype: int64

In [11]:
df1[df1['failure'] == 1]['serial_number'].value_counts()

serial_number
WD-WCC4MKDL77ZK    2
Z300KHN0           2
Z3015V4D           2
W3006E5B           2
Z300GYM0           2
                  ..
W0Q6KWV8           1
S300XP36           1
W300T2RC           1
Z302A13D           1
JK11A8B9J90WMF     1
Name: count, Length: 205, dtype: int64

In [12]:
df1[df1['failure'] == 1]['serial_number'].nunique()

205

In [5]:
df2 = pd.read_csv(data_filepath).sort_values(['serial_number', 'model', 'date'])

In [12]:
df2.shape

(3179295, 95)

In [11]:
df2.drop_duplicates().shape

(3058798, 95)

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 95 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  10000 non-null  object 
 1   serial_number         10000 non-null  object 
 2   model                 10000 non-null  object 
 3   capacity_bytes        10000 non-null  float64
 4   failure               10000 non-null  int64  
 5   smart_1_normalized    10000 non-null  int64  
 6   smart_1_raw           10000 non-null  int64  
 7   smart_2_normalized    4111 non-null   float64
 8   smart_2_raw           4111 non-null   float64
 9   smart_3_normalized    10000 non-null  int64  
 10  smart_3_raw           10000 non-null  int64  
 11  smart_4_normalized    10000 non-null  int64  
 12  smart_4_raw           10000 non-null  int64  
 13  smart_5_normalized    10000 non-null  int64  
 14  smart_5_raw           10000 non-null  int64  
 15  smart_7_normalized  

## Load data with PySpark session

In [3]:
spark = (SparkSession.builder
         .config("spark.driver.memory", "12g")
         .appName("SparkSQL").getOrCreate())

sk_data = (spark.read.option("header", "true")
           .option("inferSchema", "true")
           .csv(data_filepath))

your 131072x1 screen size is bogus. expect trouble
24/08/23 13:35:50 WARN Utils: Your hostname, cedric-yu-work resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/23 13:35:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 13:35:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/23 13:35:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sk_data.printSchema()

root
 |-- date: date (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: double (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: integer (nullable = true)
 |-- smart_1_raw: integer (nullable = true)
 |-- smart_2_normalized: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_normalized: integer (nullable = true)
 |-- smart_3_raw: integer (nullable = true)
 |-- smart_4_normalized: integer (nullable = true)
 |-- smart_4_raw: integer (nullable = true)
 |-- smart_5_normalized: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_7_normalized: integer (nullable = true)
 |-- smart_7_raw: double (nullable = true)
 |-- smart_8_normalized: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_normalized: integer (nullable = true)
 |-- smart_9_raw: integer (nullable = true)
 |-- smart_10_normalized: integer (nul

In [5]:
date_col = 'date'
target_label = 'failure'
id_cols = ['serial_number', 'model']


In [7]:
import gc
gc.collect()

477

In [10]:
sk_data.take(10)

[Row(date=datetime.date(2016, 1, 1), serial_number='MJ0351YNG9Z0XA', model='Hitachi HDS5C3030ALA630', capacity_bytes=1.4824899095664e-311, failure=0, smart_1_normalized=100, smart_1_raw=0, smart_2_normalized='135', smart_2_raw='108', smart_3_normalized=143, smart_3_raw=540, smart_4_normalized=100, smart_4_raw=14, smart_5_normalized=100, smart_5_raw=0, smart_7_normalized=100, smart_7_raw=0.0, smart_8_normalized='122', smart_8_raw='37', smart_9_normalized=97, smart_9_raw=27929, smart_10_normalized=100, smart_10_raw=0, smart_11_normalized='NA', smart_11_raw='NA', smart_12_normalized=100, smart_12_raw=14, smart_13_normalized='NA', smart_13_raw='NA', smart_15_normalized='NA', smart_15_raw='NA', smart_22_normalized='NA', smart_22_raw='NA', smart_183_normalized='NA', smart_183_raw='NA', smart_184_normalized='NA', smart_184_raw='NA', smart_187_normalized='NA', smart_187_raw='NA', smart_188_normalized='NA', smart_188_raw=0.0, smart_189_normalized='NA', smart_189_raw='NA', smart_190_normalized='

In [13]:
print(sk_data.agg({date_col: "min"}).collect()[0])
print(sk_data.agg({date_col: "max"}).collect()[0])

Row(min(date)=datetime.date(2016, 1, 1))


Row(max(date)=datetime.date(2016, 4, 29))


In [16]:
sk_data.select('model').distinct().show()

+--------------------+
|               model|
+--------------------+
|         ST9250315AS|
|         ST4000DM000|
|          ST320LT007|
|         ST3500320AS|
|      WDC WD5002ABYS|
|      WDC WD2500BEVT|
|Hitachi HDS5C3030...|
|        WDC WD60EFRX|
|       WDC WD800AAJB|
|Hitachi HDS722020...|
|      WDC WD3200LPVX|
|  TOSHIBA DT01ACA300|
|HGST HMS5C4040ALE640|
|      WDC WD3200BEKT|
| TOSHIBA MD04ABA500V|
| TOSHIBA MD04ABA400V|
|      WDC WD1600BPVT|
|Hitachi HDS723020...|
|         WDC WD800BB|
|      WDC WD2500AAJS|
+--------------------+
only showing top 20 rows



In [20]:
sk_data.agg(functions.countDistinct(sk_data.model)).collect()

[Row(count(DISTINCT model)=69)]